In [153]:
from DataStructure import *
from OrderType import *
import numpy as np

exchange = Exchange()
exchange.addAsset('TEST')

def generate_valid_price(base_mean, base_std, islong, cluster_std=3):
    while True:  
        if np.random.rand() < 0.8:  
            price = np.random.normal(loc=base_mean, scale=base_std)
        else:  
            if islong:
                cluster_center = base_mean + np.random.choice(range(-140, 0, 10)) 
                price = np.random.normal(loc=cluster_center, scale=cluster_std)
            else:
                cluster_center = base_mean + np.random.choice(range(0, 140, 10)) 
                price = np.random.normal(loc=cluster_center, scale=cluster_std)

        if price >= 1: 
            return price

for i in range(100000):
    if np.random.choice((0, 1)) == 0: 
        price = generate_valid_price(base_mean=160, base_std=30, islong=False)
        exchange.ex_trade(LimitShort(np.random.randint(10, 40), 'TEST', np.round(price, 0)))
    else:  
        price = generate_valid_price(base_mean=140, base_std=30, islong=True)
        exchange.ex_trade(LimitLong(np.random.randint(10, 40), 'TEST', np.round(price, 0)))


In [154]:
orderbook = exchange.getOrderBook('TEST')

In [155]:
long_erg = {}
short_erg = {}

def scrapePriceNodes(result: dict, node: PriceNode):
    if node == None:
        return
    result[node.price] = node.orderlist.getOrderVolume()

    if node.left != None:
        scrapePriceNodes(result, node.left)
    if node.right != None:
        scrapePriceNodes(result, node.right)

    return

In [156]:
scrapePriceNodes(long_erg, node=orderbook.long_limit.root)
scrapePriceNodes(short_erg, node=orderbook.short_limit.root)

In [157]:
combined_dict = {}


all_keys = set(short_erg.keys()).union(long_erg.keys())

for key in all_keys:
    value1 = short_erg.get(key, 0) 
    value2 = long_erg.get(key, 0) 
    combined_dict[key] = abs(value1 - value2)  

print(combined_dict)

{1.0: 1220, 2.0: 1468, 3.0: 1734, 4.0: 1629, 5.0: 1105, 6.0: 1158, 7.0: 1511, 8.0: 1892, 9.0: 2581, 10.0: 2636, 11.0: 2410, 12.0: 2180, 13.0: 1500, 14.0: 927, 15.0: 1039, 16.0: 1023, 17.0: 1921, 18.0: 2303, 19.0: 2238, 20.0: 2582, 21.0: 2658, 22.0: 1907, 23.0: 1663, 24.0: 1270, 25.0: 1161, 26.0: 1357, 27.0: 1734, 28.0: 1657, 29.0: 2078, 30.0: 2160, 31.0: 2101, 32.0: 2116, 33.0: 1575, 34.0: 1090, 35.0: 1123, 36.0: 1365, 37.0: 1453, 38.0: 1632, 39.0: 2329, 40.0: 2682, 41.0: 2043, 42.0: 1862, 43.0: 1566, 44.0: 1119, 45.0: 902, 46.0: 1510, 47.0: 1829, 48.0: 2000, 49.0: 2442, 50.0: 2110, 51.0: 2388, 52.0: 1933, 53.0: 2055, 54.0: 1325, 55.0: 1283, 56.0: 1604, 57.0: 1571, 58.0: 2016, 59.0: 3141, 60.0: 2709, 61.0: 2800, 62.0: 2760, 63.0: 1768, 64.0: 1747, 65.0: 1383, 66.0: 1600, 67.0: 2108, 68.0: 2600, 69.0: 2659, 70.0: 2898, 71.0: 2592, 72.0: 2660, 73.0: 2506, 74.0: 2492, 75.0: 2140, 76.0: 2589, 77.0: 2713, 78.0: 3371, 79.0: 3487, 80.0: 3768, 81.0: 3710, 82.0: 3499, 83.0: 3105, 84.0: 2974, 85

In [158]:
import plotly.graph_objects as go

fig = go.Figure(
    data=[
        go.Bar(
            x=list(combined_dict.keys()),  
            y=list(combined_dict.values()), 
            text=list(combined_dict.values()),
            textposition='auto',  
            marker_color='black'  
        )
    ]
)

fig.update_layout(
    title="Volume per Limit Price",
    xaxis_title="Limit Price",
    yaxis_title="Volume",
    template="plotly_white"
)

fig.show()